# Baseline : Food-drug Interaction Project

# Step 1: Load food names or compounds into a list of unique items

### Option 1 (not used): directly load csv data in Pandas dataframe

In [1]:
import pandas as pd
import numpy as np
import json

In [46]:
db_food = pd.read_csv("data/contents copy.csv", encoding = 'utf8')

In [47]:
db_food.head()

,id,orig_food_common_name
0,1,Kiwi
1,2,Onion
2,3,Onion
3,4,Chives
4,5,Cashew


In [48]:
food = db_food["orig_food_common_name"].tolist()  # somehow, converting to a set instead of list didn't work...

In [49]:
len(food)

1048575

In [87]:
foodlist = set()

for f in food:
    for term in str(f).lower().split(","):
        if len(term) >=3:  # in case some single letter or determinant is included?
            foodlist.add(term.strip())



In [68]:
i = 0
for f in foodlist:
    print f
    i += 1
    if i >20:
        break

smart balance light buttery spread
yellow
baking chocolate
swiss chard stems [red]
low salt (includes oyster
dry mix
diet strawberry kiwi
chocolate sandwich
endive [escarole]
no cholesterol
ready -to-heat
pancakes plain
standard-type
immature seeds
chocolate cake
taco with chicken
regular (10 minute)
pepeao
broiler
wholemeal
broiled


In [69]:
len(foodlist)

6143

In [70]:
with codecs.open("foodlist.txt", "w", "utf-8") as fdlist:
    for item in foodlist:
        fdlist.write(item + "\n")

### Option 2: Using Adam's pickle file (food common name as sample)

For pickling the data from foodb.ca database, see Adam's notebook **compound_food_id.ipynb**  

As a first test, we will use only the food common name (not scientific name) only. Compounds names will be added once this test passes.

In [115]:
import pickle

# food_common.pickle: Dictionary with common English food names as keys, compounds as values
test = pickle.load(open( "data/food_common.pickle", "rb"))

In [101]:
for i, item in enumerate(test.iteritems()):
    if i == 0:
        print 'food name'
        print '-'*10
    print '{0}'.format(item[0])
    if i == 10: break

food name
----------
Oregon yampah
Okra
Black mulberry
Avocado
Parsley
Elderberry
Sugar
Sweet bay
Common bean
Fig
Lard


In [104]:
foodlist_test = set(test.keys())  # Will work with a set rather than a list. Faster search for later (hash)

In [116]:
with open("foodlist.txt", "w") as fdlist:
    for item in foodlist_test:
        fdlist.write(item + "\n")

In [132]:
# Problem: there are 2 words with unusual symbols (dragee, cupuacu) and have been modified in the file and renamed
# as "foodlisr2.txt"
# Any ideas how to deal with this without changing the file manually?? Looks like codecs does not work on pickle file

foodlist = set()

with open("foodlist2.txt", "r") as fdlist2:
    for line in fdlist2:
        foodlist.add(str(line).lower().strip())


# Step 2: Filter sentences from abstract with drug keyword and food names

Note: Test with only the abstracts' first json file. Once works, we can add all 100 remaining files.

In [147]:
import pyspark
from pyspark import SparkContext
sc = SparkContext()

In [52]:
import json
# UTF-8 support
import codecs

name = "pbabstract1.json"
with codecs.open(name,"r","utf-8") as data_file:
    data = json.load(data_file)
abstractRDD = sc.parallelize(data.values())  # To load only the values and not the key (ID number)

In [29]:
abstractRDD.take(2)

[u'Heart failure still has a significant disease burden with poor outcomes worldwide despite advances in therapy. The standard therapies have been focused on blockade of renin-angiotensin-aldosterone system with angiotensin-converting enzyme inhibitors, angiotensin receptor blockers and mineralocorticoid antagonists and the sympathetic nervous system with \u03b2-blockers. The natriuretic peptide system is a potential counter-regulatory system that promotes vasodilatation and natriuresis. Angiotensin receptor neprilysin inhibitors are a new class drug capable of blocking the renin-angiotensin-aldosterone system and enhancing the natriuretic peptide system to improve neurohormonal balance. The success of the PARADIGM-HF trial with LCZ696 and its approval for heart failure treatment is likely to generate a paradigm shift. This review summarises the current knowledge of LCZ696 with a focus on pharmacology, pharmacokinetics and pharmacodynamics, mechanisms of action, clinical efficacy and s

In [53]:
# To use PubMed API
import pubmed.utils as pb

# Split abstracts to sentences
from nltk.tokenize import sent_tokenize



def splitSentences(abstract):
    sentences = sent_tokenize(abstract)
    return sentences

In [88]:
drugkeyword = "ACEI"
#foodlist = {"bla bla pla", "hahaha"}
#print type(foodlist)

In [89]:
def includeFoodCmpd(sentence, fdlist):
    if any(word in sentence for word in fdlist):
        return True
    else:
        return False

In [133]:
sentences = abstractRDD.flatMap(splitSentences) \
                       .map(lambda a: pb.ace_substitutor(a, drugkeyword)) \
                       .filter(lambda a: drugkeyword in a)\
                       .filter(lambda a: includeFoodCmpd(a, foodlist))



In [134]:
sentences.collect()

[u'the standard therapies have been focused on blockade of renin-angiotensin-aldosterone system with ACEI, angiotensin receptor blockers and mineralocorticoid antagonists and the sympathetic nervous system with \u03b2-blockers.',
 u'all patients with lower extremity peripheral arterial disease (le-pad) should benefit from recommended pharmacologic therapies including antiplatelet agents, ACEI (ACEI) inhibitors, or angiotensin receptor blockers (arbs), and hydroxy-methyl-glutaryl-coenzyme a reductase inhibitors (statins).',
 u'results were consistent across calendar periods, across subgroups of patients, and among continuing users.conclusions\xa0increases in creatinine after the start of ACEI/angiotensin receptor blocker treatment were associated with adverse cardiorenal outcomes in a graduated relation, even below the guideline recommended threshold of a 30% increase for stopping treatment.',
 u'in multivariable analyses, duration from implantation >1\xa0year (or: 2.44, 95% ci 1.12-5.3

In [93]:
sentences.count()

796

In [94]:
s = "the standard therapies have been focused on blockade of renin-angiotensin-aldosterone system with ACEI, angiotensin receptor blockers and mineralocorticoid antagonists and the sympathetic nervous system with \u03b2-blockers."

In [143]:
def findFoodItems(sentence, foods):
    for item in foods:
        if item in sentence:
            print item

In [144]:
findFoodItems(s, foodlist)

pie


In [145]:
s2 = "all patients with lower extremity peripheral arterial disease (le-pad) should benefit from recommended pharmacologic therapies including antiplatelet agents, ACEI (ACEI) inhibitors, or angiotensin receptor blockers (arbs), and hydroxy-methyl-glutaryl-coenzyme a reductase inhibitors (statins)."
findFoodItems(s2, foodlist)

pie


**Problem**: need to find words in foodlist as a complete word in the sentence and not part of a word

In [140]:
# example without "pie" item in the foodlist
foodlist2 = foodlist.copy()
foodlist2.remove("pie")

sentences = abstractRDD.flatMap(splitSentences) \
                       .map(lambda a: pb.ace_substitutor(a, drugkeyword)) \
                       .filter(lambda a: drugkeyword in a)\
                       .filter(lambda a: includeFoodCmpd(a, foodlist2))
            
sentences.collect()

[u'results were consistent across calendar periods, across subgroups of patients, and among continuing users.conclusions\xa0increases in creatinine after the start of ACEI/angiotensin receptor blocker treatment were associated with adverse cardiorenal outcomes in a graduated relation, even below the guideline recommended threshold of a 30% increase for stopping treatment.',
 u'in multivariable analyses, duration from implantation >1\xa0year (or: 2.44, 95% ci 1.12-5.31; p\u2009=\u20090.03), absence of ACEI/angiotensin ii receptor blockers (or 1.95, 95% ci 1.10-3.44; p\u2009=\u20090.02) or statins at the time of isr (or 3.12, 95% ci 1.42-6.84; p\u2009=\u20090.01), and first-generation vs first-generation des (or 5.32, 95% ci 2.82-10.10; p\u2009<\u20090.001) correlated with a higher prevalence of neoatherosclerosis.',
 u'predictors of a triggered acs were age <65\xa0years, previous angina, no previous ACEI/angiotensin 2 receptor blockers, impaired functional class, not having typical ches

In [141]:
sentences.count()

97

In [146]:
s3 = "results were consistent across calendar periods, across subgroups of patients, and among continuing users.conclusions\xa0increases in creatinine after the start of ACEI/angiotensin receptor blocker treatment were associated with adverse cardiorenal outcomes in a graduated relation, even below the guideline recommended threshold of a 30% increase for stopping treatment."
findFoodItems(s3, foodlist2)

topping


Observation: topping in food list is part of word "stopping" in the sentence and thus filtered :(

## Step 3: Classify sentences as positive or negative based on a sentiment lexicon

Sentiment lexicon used is the Harvard General Inquirer (http://www.wjh.harvard.edu/~inquirer/spreadsheet_guide.htm). It contains 1,915 positive words and 2,291 negative words and is free for research use.

In [149]:
db_sentiment = pd.read_csv("data/inquirerbasic.csv", encoding = 'utf8')

/Applications/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (63,108,109,110,176) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [150]:
db_sentiment.head()

,Entry,Source,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,...,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,Othtags,Defined
0,A,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET ART,| article: Indefinite singular article--some o...
1,ABANDON,H4Lvd,NaN,Negativ,NaN,NaN,Ngtv,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|
2,ABANDONMENT,H4,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,|
3,ABATE,H4Lvd,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|
4,ABATEMENT,Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,NaN


In [153]:
# Look at all data fields available
for column in db_sentiment.columns:
    print column

Entry
Source
Positiv
Negativ
Pstv
Affil
Ngtv
Hostile
Strong
Power
Weak
Submit
Active
Passive
Pleasur
Pain
Feel
Arousal
EMOT
Virtue
Vice
Ovrst
Undrst
Academ
Doctrin
Econ@
Exch
ECON
Exprsv
Legal
Milit
Polit@
POLIT
Relig
Role
COLL
Work
Ritual
SocRel
Race
Kin@
MALE
Female
Nonadlt
HU
ANI
PLACE
Social
Region
Route
Aquatic
Land
Sky
Object
Tool
Food
Vehicle
BldgPt
ComnObj
NatObj
BodyPt
ComForm
COM
Say
Need
Goal
Try
Means
Persist
Complet
Fail
NatrPro
Begin
Vary
Increas
Decreas
Finish
Stay
Rise
Exert
Fetch
Travel
Fall
Think
Know
Causal
Ought
Perceiv
Compare
Eval@
EVAL
Solve
Abs@
ABS
Quality
Quan
NUMB
ORD
CARD
FREQ
DIST
Time@
TIME
Space
POS
DIM
Rel
COLOR
Self
Our
You
Name
Yes
No
Negate
Intrj
IAV
DAV
SV
IPadj
IndAdj
PowGain
PowLoss
PowEnds
PowAren
PowCon
PowCoop
PowAuPt
PowPt
PowDoct
PowAuth
PowOth
PowTot
RcEthic
RcRelig
RcGain
RcLoss
RcEnds
RcTot
RspGain
RspLoss
RspOth
RspTot
AffGain
AffLoss
AffPt
AffOth
AffTot
WltPt
WltTran
WltOth
WltTot
WlbGain
WlbLoss
WlbPhys
WlbPsyc
WlbPt
WlbTot
EnlGain
EnlLo

**Note:** some columns seem quite interesting for analysing relationship other than simply positive or negative sentiment (e.g. "causal", etc.). For the baseline, we will only use the "positive" and "negative" columns

In [156]:
# Filter only the words labeled positive or negative
positive = db_sentiment[db_sentiment.Positiv == "Positiv"].Entry.map(lambda x: x.lower()).tolist()
negative = db_sentiment[db_sentiment.Negativ == "Negativ"].Entry.map(lambda x: x.lower()).tolist()

In [157]:
positive[:4]

[u'abide', u'ability', u'able', u'abound']

In [158]:
negative[:4]

[u'abandon', u'abandonment', u'abate', u'abdicate']

In [159]:
# Transform list into sets for faster search
positive = set(positive)
negative = set(negative)

Below is an attempt to classify whether a sentence is positive or negative.  
Note the main weaknesses:  
1. It is "positively" biased for now since looks at the positive words first and if it finds it, then it immediately returns positive. Thus, it may not look at the entire sentence in case of both positive or negative words.  
2. Negation of a positive word is not taken into account

In [160]:
def includeSentiment(sentence, poslist, neglist): 
    ''' Classify sentence as positive or negative based on first word found
        in the lexicon
    '''
    if any(word in sentence for word in poslist):
        return ("positive", sentence)
    elif any(word in sentence for word in neglist):
        return ("negative", sentence)
    else:
        return ("neutral", sentence)

In [161]:
# Add sentiment as key in the RDD
sentiments = sentences.map(lambda a: includeSentiment(a, positive, negative))


In [166]:
# Peek at 2 first lines
sentiments.take(2)

[('positive',
  u'results were consistent across calendar periods, across subgroups of patients, and among continuing users.conclusions\xa0increases in creatinine after the start of ACEI/angiotensin receptor blocker treatment were associated with adverse cardiorenal outcomes in a graduated relation, even below the guideline recommended threshold of a 30% increase for stopping treatment.'),
 ('positive',
  u'in multivariable analyses, duration from implantation >1\xa0year (or: 2.44, 95% ci 1.12-5.31; p\u2009=\u20090.03), absence of ACEI/angiotensin ii receptor blockers (or 1.95, 95% ci 1.10-3.44; p\u2009=\u20090.02) or statins at the time of isr (or 3.12, 95% ci 1.42-6.84; p\u2009=\u20090.01), and first-generation vs first-generation des (or 5.32, 95% ci 2.82-10.10; p\u2009<\u20090.001) correlated with a higher prevalence of neoatherosclerosis.')]

In [177]:
# 
pos_sentiments = sentiments.lookup("positive")
neg_sentiments = sentiments.lookup("negative")

In [183]:
# Examples of sentences with positive sentiment lexicon
pos_sentiments[:10]

[u'results were consistent across calendar periods, across subgroups of patients, and among continuing users.conclusions\xa0increases in creatinine after the start of ACEI/angiotensin receptor blocker treatment were associated with adverse cardiorenal outcomes in a graduated relation, even below the guideline recommended threshold of a 30% increase for stopping treatment.',
 u'in multivariable analyses, duration from implantation >1\xa0year (or: 2.44, 95% ci 1.12-5.31; p\u2009=\u20090.03), absence of ACEI/angiotensin ii receptor blockers (or 1.95, 95% ci 1.10-3.44; p\u2009=\u20090.02) or statins at the time of isr (or 3.12, 95% ci 1.42-6.84; p\u2009=\u20090.01), and first-generation vs first-generation des (or 5.32, 95% ci 2.82-10.10; p\u2009<\u20090.001) correlated with a higher prevalence of neoatherosclerosis.',
 u'predictors of a triggered acs were age <65\xa0years, previous angina, no previous ACEI/angiotensin 2 receptor blockers, impaired functional class, not having typical ches

In [179]:
# Examples of sentences with negative sentiment lexicon
neg_sentiments[:2]

[u'the late-eluting fraction (f4) of either whey or caseins exhibited greater ACEI inhibition.',
 u'commonly used drugs including ACEI and statins increase a risk of hyperkalemia.']

In [180]:
def findSentiment(sentence, sentiment, poslist, neglist):
    '''Print out the lexicon word that classified the sentence as positive or negative
    '''
    if sentiment == "positive":
        lexicon = poslist
    else:
        lexicon = neglist
    for word in lexicon:
        # Note: had to use this "try/except" since there was an unicode ascii error... any ways
        # to fix this without this try/except? if left the same, then we won't be able to see
        # some sentiment word in some sentences.
        try:
            if word in sentence:
                print word
        except:
            next

In [184]:
s5 = "digestion of isolated whey proteins and caseins of goat milk by gastric pepsin generated soluble hydrolysates exhibiting significant inhibition of ACEI compared to weak inhibition by undigested proteins"
findFoodItems(s5, foodlist)
findSentiment(s5, "positive", positive, negative)

casein
oat
whey
pro
generate
significant


In [182]:
s6 = "the late-eluting fraction (f4) of either whey or caseins exhibited greater ACEI inhibition"
findFoodItems(s6, foodlist)
findSentiment(s6, "negative", positive, negative)

casein
whey
inhibit
inhibition


In [185]:
def findTags(sentence, sentiment, foods, poslist, neglist):
    ''' Returns the tags of the sentence 
        Both lexicon word that classified the sentence as positive or negative and food name
    '''
    
    sent = []
    food = []
    
    if sentiment == "positive":
        lexicon = poslist
    else:
        lexicon = neglist
    for word in lexicon:
        # Note: had to use this "try/except" since there was an unicode ascii error... any ways
        # to fix this without this try/except? if left the same, then we won't be able to see
        # some sentiment word in some sentences.
        try:
            if word in sentence:
                sent.append(word)
        except:
            next
    for f in foods:
        if f in sentence:
            food.append(f)
    return [sent, food, sentence]

In [197]:
# Save results in a text file
# Note: this could have been also done in Spark! But felt lazy to code... feel free to try!

In [196]:
with open("data/Positive.txt", "w") as pos:
    for sentence in pos_sentiments:
        tags = findTags(sentence, "positive", foodlist, positive, negative)
        pos.writelines(str(tags)+ "\n")
    
    

In [195]:
with open("data/Negative.txt", "w") as pos:
    for sentence in pos_sentiments:
        tags = findTags(sentence, "negative", foodlist, positive, negative)
        pos.writelines(str(tags)+ "\n")